## DuneAI segmentation in XNAT

In [1]:
# First, run the following command from terminal: 
# /workspace/admin/envs/duneai/etc/activate.d/init.sh
# then select 'duneai' kernel from available kernels. Restart notebook if not listed.

### 1. Configure data

In [1]:
import os, subprocess, sys
#local imports
pymipl_path = os.path.abspath(os.path.join('../../','pymipl'))
sys.path.append(pymipl_path)
from dicom_sort import *
from pathlib import Path

project='RIDER-LUNG-CT'
path=f'/data/projects/{project}/experiments'
d=analyze_dir(path)


/data/projects/RIDER-LUNG-CT/experiments/09-20-2006-1-NA-96508
/data/projects/RIDER-LUNG-CT/experiments/09-20-2006-1-NA-96508_SEG


### 2. Detect structural CT scans in data.

In [2]:
from pathlib import Path
ct_scans=[]
for exp in d['children']:
    #print (Path(exp['path']).stem)
    exp_label=Path(exp['path']).stem
    for subdir1 in exp['children']:        
        if Path(subdir1['path']).stem == 'SCANS': 
            #print(subdir1['path'])
            for scan in subdir1['children']:
                scan_label=Path(scan['path']).stem
                #print (scan_label)
                try:
                    #print(scan['children'][0]['children'][0]['SOPClass'])
                    if scan['children'][0]['children'][0]['SOPClass']=='CTImageStorage':
                        #print('CT image:', scan_label)
                        ct_scans+=[scan['children'][0]['children'][0]]
                except Exception as e:
                    pass
                    
print(ct_scans)
                    

[{'PatName': 'RIDER-1129164940', 'SeriesDescription': None, 'SOPClass': 'CTImageStorage', 'SeriesInstanceUID': '1.3.6.1.4.1.9328.50.1.48441840081578419409180519840073808100', 'path': '09-20-2006-1-NA-96508/SCANS/8/DICOM/1-187.dcm', 'level': 5}, {'PatName': 'RIDER-1129164940', 'SeriesDescription': None, 'SOPClass': 'CTImageStorage', 'SeriesInstanceUID': '1.3.6.1.4.1.9328.50.1.83304264089411327530730818890072724533', 'path': '09-20-2006-1-NA-96508/SCANS/4/DICOM/1-187.dcm', 'level': 5}]


### 3. Create batch file.

In [34]:
import datetime

rerun_segmentation=False
rerun_qc=False
root_dir=Path("/workspace/admin")
outdir=root_dir / project
pymipl_dir=root_dir / "pymipl"
duneai_dir=root_dir / "DuneAI/AutomaticSegmentationScript"

dt=datetime.datetime.now().strftime("%Y%m%d_%H%M")
batch_file=outdir / f"batch_{dt}.sh"
print(batch_file)
! echo "#!/bin/bash" > {batch_file}
for scan in ct_scans:
    dcm_path=(Path(path) / scan['path']).parent
    scan_id=Path(scan['path']).parent.parent.stem
    exp_label=Path(scan['path']).parent.parent.parent.parent.stem
    #print(exp_label)
    subject=scan['PatName']
    out_path=outdir / f"{subject}/{exp_label}/{scan_id}/ct"
    #!echo "outdir: {out_path}"
    out_path.parent.mkdir(parents=True,exist_ok=True)
    #create nifti
    print (f"python {(pymipl_dir / 'test_rt-utils.py')} {dcm_path} {out_path}")
    #convert to NIFTI if not already
    if not Path(out_path.as_posix()+'_struct.nii').exists():
        ! echo "python {(pymipl_dir / 'test_rt-utils.py')} {dcm_path} {out_path}" >> {batch_file}    
    if rerun_segmentation or not (out_path.parent / f"{scan_id}_duneai/DL_mask.nii").exists():
        ! echo "python {(duneai_dir / 'segmentation-cl.py')} {out_path}_struct.nii {out_path.parent} {out_path.parent}" >> {batch_file}
    duneai_out=out_path.parent / f"{scan_id}_duneai"
    if not rerun_qc or not (duneai_out / three_plane_view.png).exists():
        ! echo "python {(pymipl_dir / 'show_overlays.py')} -o {duneai_out / 'qc.png'} {duneai_out / 'image.nii'} {duneai_out / 'lung_mask.nii'} {duneai_out / 'DL_mask.nii'} " >> {batch_file}

! chmod +x {batch_file}        
        

/workspace/admin/RIDER-LUNG-CT/batch_20251117_2151.sh
python /workspace/admin/pymipl/test_rt-utils.py /data/projects/RIDER-LUNG-CT/experiments/09-20-2006-1-NA-96508/SCANS/8/DICOM /workspace/admin/RIDER-LUNG-CT/RIDER-1129164940/09-20-2006-1-NA-96508/8/ct
python /workspace/admin/pymipl/test_rt-utils.py /data/projects/RIDER-LUNG-CT/experiments/09-20-2006-1-NA-96508/SCANS/4/DICOM /workspace/admin/RIDER-LUNG-CT/RIDER-1129164940/09-20-2006-1-NA-96508/4/ct


### 4. Review QC.

In [37]:
import ipywidgets as widgets
from IPython.display import display
from pathlib import Path
from PIL import Image
import io

# --------------------------------------------------------------
# 1. Collect all qc.png files produced by the previous step
# --------------------------------------------------------------

qc_entries = []

for scan in ct_scans:
    scan_id = Path(scan['path']).parent.parent.stem
    exp_label = Path(scan['path']).parent.parent.parent.parent.stem
    subject = scan['PatName']
    
    out_path = outdir / f"{subject}/{exp_label}/{scan_id}/ct"
    duneai_out = out_path.parent / f"{scan_id}_duneai"
    qc_path = duneai_out / "qc.png"

    if qc_path.exists():
        qc_entries.append(
            {
                "subject": subject,
                "exp": exp_label,
                "scan_id": scan_id,
                "path": qc_path,
            }
        )

# Safety check
if not qc_entries:
    raise ValueError("No qc.png files found.")

# Sort for stable ordering
qc_entries = sorted(qc_entries, key=lambda x: (x["subject"], x["exp"], x["scan_id"]))

# --------------------------------------------------------------
# 2. Widgets
# --------------------------------------------------------------

idx = 0

label = widgets.Text(
    value="",
    description="Status:",
    layout=widgets.Layout(width="600px"),
)

image_widget = widgets.Output()

btn_prev = widgets.Button(description="Prev", button_style='')
btn_next = widgets.Button(description="Next", button_style='')

# --------------------------------------------------------------
# 3. Display update function
# --------------------------------------------------------------

def show_entry(i):
    entry = qc_entries[i]
    label.value = f"Subject: {entry['subject']}, experiment: {entry['exp']}, scan: {entry['scan_id']}"

    img_path = entry["path"]
    img = Image.open(img_path)

    image_widget.clear_output(wait=True)
    with image_widget:
        display(img)

def on_prev(_):
    global idx
    idx = (idx - 1) % len(qc_entries)
    show_entry(idx)

def on_next(_):
    global idx
    idx = (idx + 1) % len(qc_entries)
    show_entry(idx)

btn_prev.on_click(on_prev)
btn_next.on_click(on_next)

# --------------------------------------------------------------
# 4. Arrange UI layout
# --------------------------------------------------------------

buttons = widgets.HBox([btn_prev, btn_next])
ui = widgets.VBox([label, image_widget, buttons])

display(ui)

# Initial display
show_entry(idx)
